In [1]:
import os
import json
from datetime import datetime
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import box, mapping
import pystac
import xml.etree.ElementTree as ET
import sys
sys.path.append('/home/vishnu/corestack_STAC')
from constants import DEFAULT_START_DATE, DEFAULT_END_DATE


/home/vishnu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [ ]:
input_file = "/home/vishnu/corestack_STAC/data/swb2_saraikela-kharsawan_gobindpur.geojson"
qgis_style_path = "/home/vishnu/corestack_STAC/data/swb_style.qml"
data_dir = os.path.dirname(input_file)


In [ ]:
with rasterio.open(input_file) as src:
    bounds = src.bounds
    bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
    geometry = mapping(box(*bbox))
    epsg = src.crs.to_epsg()


RasterioIOError: '/home/vishnu/corestack_STAC/data/swb2_saraikela-kharsawan_gobindpur.geojson' not recognized as being in a supported file format.

In [ ]:
filename = os.path.basename(input_file)
parts = filename.split('_')
try:
    start_date = datetime.strptime(parts[2], "%Y-%m-%d")
    end_date = datetime.strptime(parts[3], "%Y-%m-%d")
except Exception as e:
    print(f"Failed to extract dates from filename '{filename}': {e}")
    start_date = DEFAULT_START_DATE
    end_date = DEFAULT_END_DATE


In [ ]:
output_dir = "/home/vishnu/corestack_STAC/output_catalog_vector"
item_id = "gobindpur-geojson"
item_dir = os.path.join(output_dir, item_id)
os.makedirs(item_dir, exist_ok=True)

catalog = pystac.Catalog(
    id="gobindpur-geojson-catalog",
    description=""
)

item = pystac.Item(
    id=item_id,
    geometry=geometry,
    bbox=bbox,
    datetime=start_date,
    properties={
        "start_datetime": start_date.isoformat() + "Z",
        "end_datetime": end_date.isoformat() + "Z",
        "proj:epsg": epsg,
        "proj:bbox": bbox
    },
    stac_extensions=[
        "https://stac-extensions.github.io/projection/v1.0.0/schema.json"
    ]
)


In [ ]:
item.add_asset(
    key="raster-data",
    asset=pystac.Asset(
        href="home/vishnu/corestack_STAC/data/" + os.path.basename(input_file),
        media_type=pystac.MediaType.GEOTIFF,
        roles=["data"],
        title="Gobindpur LULC Raster (GeoTIFF)"
    )
)


In [24]:
if os.path.exists(qgis_style_path):
    print("QML file found. Parsing classification...")
    lulc_classes = parse_qml_to_classes(qgis_style_path)

    item.add_asset(
        key="qgis-style",
        asset=pystac.Asset(
            href="home/vishnu/corestack_STAC/data/style_file.qml",
            media_type="application/xml",
            roles=["style"],
            title="QGIS Style File"
        )
    )

    item.properties["classification:classes"] = lulc_classes

    legend_path = os.path.join(data_dir, "legend.json")
    with open(legend_path, "w") as f:
        json.dump(lulc_classes, f, indent=2)

    item.add_asset(
        key="legend",
        asset=pystac.Asset(
            href="home/vishnu/corestack_STAC/data/legend.json",
            media_type="application/json",
            roles=["legend"],
            title="LULC Legend"
        )
    )
else:
    print("QML file not found. Skipping classification section.")


QML file found. Parsing classification...


In [ ]:
thumb_path = os.path.join(data_dir, "thumbnail.png")
with rasterio.open(input_file) as src:
    array = src.read(1)

plt.figure(figsize=(3, 3))
plt.axis('off')
plt.imshow(array, cmap='tab20')
plt.savefig(thumb_path, bbox_inches='tight', pad_inches=0)
plt.close()

item.add_asset(
    key="thumbnail",
    asset=pystac.Asset(
        href="home/vishnu/corestack_STAC/data/thumbnail.png",
        media_type="image/png",
        roles=["thumbnail"],
        title="Thumbnail Preview"
    )
)


In [26]:
catalog.add_item(item)
catalog.normalize_hrefs(output_dir)
catalog.make_all_asset_hrefs_relative()
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)


default_item_path = os.path.join(item_dir, "item.json")
custom_item_path = os.path.join(item_dir, f"{item_id}.json")
if os.path.exists(default_item_path):
    os.rename(default_item_path, custom_item_path)

print("\nSTAC catalog created with:")
print(" Dates from filename")
print(" classification:classes from QML")
print(" Thumbnail preview")
print(" catalog.json:", os.path.join(output_dir, "catalog.json"))
print("item:", custom_item_path)



STAC catalog created with:
 Dates from filename
 classification:classes from QML
 Thumbnail preview
 catalog.json: /home/vishnu/corestack_STAC/output_catalog_lulc/catalog.json
item: /home/vishnu/corestack_STAC/output_catalog_lulc/gobindpur-lulc/gobindpur-lulc.json
